# Simon UN Data

In [1]:
%load_ext lab_black
%load_ext autoreload
%load_ext watermark

In [2]:
%autoreload 2

In [3]:
%watermark -ntz -p pandas -a Simon-Lee-UK -u -d -t -z

Author: Simon-Lee-UK

Last updated: 2021-01-20 21:29:15GMT

pandas: 1.2.1



In [4]:
import sys
from pathlib import Path
import pandas as pd

repo_root = (
    Path.cwd().resolve().parent.parent.parent
)  # ToDo: get 'pyprojroot' onto the env and use that instead
split_data = repo_root / "raw_data"

In [5]:
repo_root

PosixPath('/Users/Simon/Documents/Python_Projects/see-you-data/UNInternationalEnergyData')